In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from ops import Operations
from session import *
from data_gen import *
import pickle
from functools import reduce

modelName = 'RNN'
old = False
func = 'np_avg_val-5ops'
name = 'one_both_hardmaxes_have_sat_sat_grad_out/state_size_300-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_4-norm_True-softmax_sat_800-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0-seed_94169'
path = '/home/user/projects/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(5,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W3_mem:0' shape=(5, 4) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b3_mem:0' shape=(4,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(5,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W3:0' shape=(5, 4) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b3:0' shape=(4,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/c

In [ ]:
def print_errors(error_lst, name):
    print("")
    print("Total", reduce((lambda x, y: x + y), error_lst))
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [2]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [3]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [ ]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")

In [5]:
#print_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"])

Error for this matrix is 1.45150406561e-28

##Elem-0--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-1--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-2--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-3--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-4--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-5--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-6--#############################################################
[ tf_add ] [ tf_add ] [ tf_add ] [ tf_divide ] [ tf_add ] 
##Elem-7--#############################

In [6]:
#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

Error for this matrix is 1.45150406561e-28

##Elem-0--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-1--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-2--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-3--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-4--#############################################################
[ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-5--#############################################################
[ tf_stall ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] [ tf_inpt_len ] 
##Elem-6--###############################

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

In [ ]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res['last_hardmax_state_train_mem']

In [ ]:
cfg['num_features']

In [7]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res_pred = predict_form_sess(model, cfg, x_test[0:100,], res['last_hardmax_state_train'],res['last_hardmax_state_train_mem'], path = model_path, mode="hard")

INFO:tensorflow:Restoring parameters from /home/user/projects/neural_program_synthesis/models/summaries/RNN/np_avg_val-5ops/one_both_hardmaxes_have_sat_sat_grad_out/state_size_300-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_4-norm_True-softmax_sat_800-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0-seed_94169//model/-510


In [9]:
res_pred

[array([[-16.036,   0.   ,   0.   ,   0.   ],
        [ 27.502,   0.   ,   0.   ,   0.   ],
        [ 52.418,   0.   ,   0.   ,   0.   ],
        [-28.386,   0.   ,   0.   ,   0.   ],
        [ 46.806,   0.   ,   0.   ,   0.   ],
        [-17.814,   0.   ,   0.   ,   0.   ],
        [ 31.01 ,   0.   ,   0.   ,   0.   ],
        [ 33.416,   0.   ,   0.   ,   0.   ],
        [ -3.253,   0.   ,   0.   ,   0.   ],
        [ -6.009,   0.   ,   0.   ,   0.   ],
        [ 10.124,   0.   ,   0.   ,   0.   ],
        [-18.129,   0.   ,   0.   ,   0.   ],
        [-15.935,   0.   ,   0.   ,   0.   ],
        [-38.142,   0.   ,   0.   ,   0.   ],
        [ -2.284,   0.   ,   0.   ,   0.   ],
        [ 29.673,   0.   ,   0.   ,   0.   ],
        [ 22.864,   0.   ,   0.   ,   0.   ],
        [ 32.752,   0.   ,   0.   ,   0.   ],
        [-47.395,   0.   ,   0.   ,   0.   ],
        [ 10.248,   0.   ,   0.   ,   0.   ],
        [ 26.164,   0.   ,   0.   ,   0.   ],
        [ 29.636,   0.   ,   0.   

In [8]:
x_test[0:100,]

array([[-88.459,  36.053,   5.965, -17.703],
       [ 28.399, -12.488,  33.972,  60.124],
       [ 89.814,  75.709,  95.046, -50.897],
       [-45.428, -80.059, -22.983,  34.926],
       [ 82.096,  58.668,  44.3  ,   2.159],
       [ 16.587,   7.113, -69.58 , -25.374],
       [ 24.689,  33.067,  25.018,  41.266],
       [ 58.459,  21.332,  44.759,   9.113],
       [ 89.187, -76.12 , -83.901,  57.824],
       [-43.279, -76.765,  65.722,  30.286],
       [-41.012,  81.   , -62.838,  63.346],
       [-88.171, -24.372,  47.13 ,  -7.104],
       [ 14.746,   8.844,   9.757, -97.087],
       [-92.248,  -1.37 , -71.026,  12.074],
       [ -5.267,  49.927, -31.957, -21.84 ],
       [-30.399,  57.653,  43.136,  48.302],
       [-31.95 ,  95.744,   4.232,  23.428],
       [ 49.486,  23.054,  31.646,  26.824],
       [-80.268, -57.051, -39.364, -12.898],
       [ 31.15 ,  97.644, -21.4  , -66.403],
       [ 13.399,  -7.78 ,  98.059,   0.979],
       [ 52.645,  50.21 ,  10.111,   5.577],
       [ 4

In [ ]:
res_pred

In [ ]:
len(res["softmaxes_traind_train"])